In [22]:
import arivale_data_interface as adi
import numpy, pandas, glob, random, seaborn
import matplotlib.pyplot as plt
import seaborn as sns
import scipy.stats
from matplotlib.lines import Line2D
from matplotlib import colors
import rpy2.robjects as R
from rpy2.robjects import numpy2ri, pandas2ri
numpy2ri.activate()
pandas2ri.activate()
import sklearn.linear_model
import statsmodels
import statsmodels.api as sm
from statsmodels.formula.api import ols
from statsmodels.stats import multitest

# get phenotype

In [12]:
# define cohort and taxonomy level
cohort = 'both'
taxa_level = 'genus'
bmi_or_not = 'bmi'
classification = 'rdp'
classification_method = classification + '_classification'

## metabolomics

In [13]:
# get phenotype and covariates
metab_pheno = pandas.read_csv('../gxe/input_pheno/both_metab_10202019.pheno'.format(cohort), sep = '\t', index_col = [0,1])
metab_covar = pandas.read_csv('../gxe/input_pheno/both_metab_10202019.covar'.format(cohort), sep = '\t', index_col = [0,1])
metab_metadata = pandas.read_csv('../gxe/input_pheno/all_metab_11102019.pheno.metadata', sep = '\t')
unknown_metabolite = metab_metadata[metab_metadata['metabolite'].str.contains('X -')]['metabolite_id'].astype(str).unique()
all_metabolites = metab_pheno.columns
metabolomics_metadata_dict = metab_metadata.set_index('metabolite_id')['metabolite'].to_dict()

metab_covar['age'] = (metab_covar['age'] - metab_covar['age'].mean())/metab_covar['age'].std()
metab_covar['age_sq'] = metab_covar['age']**2
metab_covar['sex_age'] = metab_covar['sex'].apply({'M':1, 'F':0}.get) * metab_covar['age']
metab_covar['sex_age_sq'] = metab_covar['sex'].apply({'M':1, 'F':0}.get) * metab_covar['age_sq']

# get covariates
metab_covariates = ['age', 'age_sq','sex_age', 'sex_age_sq', 'sex', 'season', 'batch', 'PC1', 'PC2','PC3','PC4','PC5', 'bmi']
metab_factors = ['sex','season','batch']

# merge phenotype data with covariates
metab_combined = metab_pheno.join(metab_covar[metab_covariates])
metab_combined.index = metab_combined.index.droplevel(0)

# adjust for covariates
metab_adj = metab_combined[[]].copy()

In [14]:
metab_rename = {metab:'X{0}'.format(metab) for metab in all_metabolites}
metab_combined = metab_combined.rename(columns = metab_rename)

for metabolite in all_metabolites:
    out = ols('X{0} ~ age + age_sq + sex*age + sex*age_sq + sex + batch + bmi + PC1 + PC2 + PC3 + PC4 + PC5'.format(metabolite), metab_combined).fit()
    metab_adj[metabolite] = out.resid
    
# get ONLY significant associations
sig_metab_assoc = pandas.read_csv('../gxe/validate_gwas/clumped_variants/metab/fastgwa/clumped_10perc_second_pass_list.txt',
               sep = '\t', header = None)
sig_metab_assoc.columns = ['rsid', 'Phenotype']
# sig_metab_assoc = sig_metab_assoc[~sig_metab_assoc['Phenotype'].str.contains('X999')]
sig_metab_assoc = numpy.unique(sig_metab_assoc['Phenotype'].str.replace('X','').astype(str).values)


## microbiome

In [15]:
# get all of the taxa tables
micro_pheno_sets = {}
micro_metadata = []
for t_level in taxa_level.split('_'):
    micro_pheno_sets[t_level] = pandas.read_csv('../microbiome_gwas/phyloseq/asv_analysis/{3}/{1}/{0}_asv_{1}_clr_adjusted_{2}_10272020.pheno'.format(cohort, t_level, bmi_or_not, classification_method), sep = '\t', index_col = [0,1])
    tax_metadata = pandas.read_csv('../microbiome_gwas/phyloseq/asv_analysis/{1}/{0}/renamed_asv_{0}_full_taxonomy_table_10282020.csv'.format(t_level, classification_method))
    tax_metadata['tax_id'] = tax_metadata['tax_id'] + '_' + t_level
    micro_metadata.append(tax_metadata)

# pop the first level out
t_level_orig, micro_pheno = micro_pheno_sets.popitem()
micro_pheno = micro_pheno.rename(columns= {i:'_'.join([i,t_level_orig]) for i in micro_pheno.columns})

# iterate through the rest
for t_level in micro_pheno_sets.keys():
    temp_micro_pheno = micro_pheno_sets[t_level]
    temp_micro_pheno = temp_micro_pheno.rename(columns= {j:'_'.join([j,t_level]) for j in temp_micro_pheno.columns})
    micro_pheno = micro_pheno.join(temp_micro_pheno)
    
# get metadata
micro_metadata = pandas.concat(micro_metadata)
micro_metadata = micro_metadata.append({'tax_id':'alpha_div', 'taxonomy':'Shannon alpha diversity'}, ignore_index=True)
micro_metadata.to_csv('./joint_pheno/{0}_{1}.metadata'.format(taxa_level, classification_method), index = None, sep = '\t')

# remove duplicate alpha diversity
alpha_div_columns = micro_pheno.columns[micro_pheno.columns.str.contains('alpha_div')]
micro_pheno['alpha_div'] = micro_pheno.loc[:,alpha_div_columns].mean(axis = 1)
micro_pheno = micro_pheno.drop(columns=alpha_div_columns)
taxas = micro_pheno.columns.values

print('num taxas to test: {0}'.format(len(taxas)))

micro_metab_subset_pheno = micro_pheno[micro_pheno.index.isin(metab_pheno.index)]

num taxas to test: 64


# microbiome - metabolite associations

In [16]:
# merge adjusted phenotypes
micro_metab_adj = micro_metab_subset_pheno.join(metab_adj)
micro_metab_adj.to_csv('./joint_pheno/{0}_{1}_{2}_micro_metab_adj.pheno'.format(cohort, taxa_level, classification))

micro_metab_unadj = micro_metab_subset_pheno.join(metab_pheno)
micro_metab_unadj.to_csv('./joint_pheno/{0}_{1}_{2}_micro_metab_unadj.pheno'.format(cohort, taxa_level, classification))
metab_covar.to_csv('./joint_pheno/{0}_{1}_{2}_micro_metab_unadj.covar'.format(cohort, taxa_level, classification))
len(micro_metab_adj)

1163

## association analysis

In [17]:
all_metabolite_all_taxas_assoc = []
for metabolite in set(all_metabolites) - set(unknown_metabolite):
    for taxa in set(taxas):
        temp = micro_metab_adj[[metabolite, taxa]].dropna()
        
        slope, intercept, r, pval, std = scipy.stats.linregress(temp[metabolite].values, temp[taxa].values)

        all_metabolite_all_taxas_assoc.append([metabolite, taxa, pval, r])
        
print('num of comparisons:{0}'.format(len(all_metabolite_all_taxas_assoc)))

num of comparisons:50368


In [18]:
# get annotations
all_metabolite_all_taxas_assoc_df = pandas.DataFrame(all_metabolite_all_taxas_assoc).drop_duplicates()
all_metabolite_all_taxas_assoc_df.columns = ['metabolite_id','tax_id', 'pval', 'r']
all_metabolite_all_taxas_assoc_annotated = all_metabolite_all_taxas_assoc_df.merge(micro_metadata)
all_metabolite_all_taxas_assoc_annotated = all_metabolite_all_taxas_assoc_annotated.merge(metab_metadata[['metabolite_id','metabolite']].astype(str))

In [19]:
# do FDR correction
fdr_pass, pval_corrected, sidak, bonf = statsmodels.stats.multitest.multipletests(all_metabolite_all_taxas_assoc_annotated['pval'], method='fdr_bh')

all_metabolite_all_taxas_assoc_annotated['fdr_pass'] = fdr_pass
all_metabolite_all_taxas_assoc_annotated['pval_corrected'] = pval_corrected

all_metabolite_all_taxas_assoc_annotated_fdr = all_metabolite_all_taxas_assoc_annotated[all_metabolite_all_taxas_assoc_annotated['fdr_pass'] == True]
sig_metabolite_all_taxas_assoc_annotated_fdr = all_metabolite_all_taxas_assoc_annotated_fdr[all_metabolite_all_taxas_assoc_annotated_fdr['metabolite_id'].isin(numpy.unique(sig_metab_assoc))]

all_metabolite_all_taxas_assoc_annotated_fdr.to_csv('./assoc_results/{0}_all_metabolite_all_{1}_{3}_assoc_fdr_corr_{2}.csv'.format(cohort, taxa_level, bmi_or_not, classification), index = None)
sig_metabolite_all_taxas_assoc_annotated_fdr.to_csv('./assoc_results/{0}_sig_metabolite_all_{1}_{3}_assoc_fdr_corr_{2}.csv'.format(cohort, taxa_level, bmi_or_not, classification), index = None)
print(len(all_metabolite_all_taxas_assoc_annotated_fdr))
print(len(all_metabolite_all_taxas_assoc_annotated_fdr['metabolite_id'].unique()))
print(len(sig_metabolite_all_taxas_assoc_annotated_fdr))
print(len(sig_metabolite_all_taxas_assoc_annotated_fdr['metabolite_id'].unique()))

1968
417
459
146


In [14]:
# adjust for alpha diversity

all_all_assoc_adj_diversity = []
for metabolite in set(all_metabolites) - set(unknown_metabolite):
    temp = micro_metab_adj[[metabolite, 'alpha_div']].dropna()
    slope, intercept, r, pval, std = scipy.stats.linregress(temp[metabolite].values, temp['alpha_div'].values)
    all_all_assoc_adj_diversity.append([metabolite, 'alpha_div', pval, r])
    
    for taxa in set(taxas) - set(['alpha_div']):
        temp = micro_metab_adj[[metabolite, taxa, 'alpha_div']].dropna()
        
        div_resid = sm.OLS(temp[metabolite].values, sm.add_constant(temp[['alpha_div']].values)).fit().resid

        slope, intercept, r, pval, std = scipy.stats.linregress(div_resid, temp[taxa].values)

        all_all_assoc_adj_diversity.append([metabolite, taxa, pval, r])
        
        
print('num of comparisons:{0}'.format(len(all_all_assoc_adj_diversity)))

num of comparisons:67682


In [15]:
# get annotations alpha adjusted
all_all_assoc_adj_diversity_df = pandas.DataFrame(all_all_assoc_adj_diversity,
                                                  columns = ['metabolite_id','tax_id','pval', 'r']).drop_duplicates()
all_all_assoc_adj_diversity_annotated = all_all_assoc_adj_diversity_df.merge(micro_metadata)
all_all_assoc_adj_diversity_annotated = all_all_assoc_adj_diversity_annotated.merge(metab_metadata[['metabolite_id','metabolite']].astype(str))

In [16]:
fdr_pass, pval_corrected, sidak, bonf = statsmodels.stats.multitest.multipletests(all_all_assoc_adj_diversity_annotated['pval'], method='fdr_bh')

all_all_assoc_adj_diversity_annotated['fdr_pass'] = fdr_pass
all_all_assoc_adj_diversity_annotated['pval_corrected'] = pval_corrected

all_all_assoc_adj_diversity_annotated_fdr = all_all_assoc_adj_diversity_annotated[all_all_assoc_adj_diversity_annotated['fdr_pass'] == True]
sig_metabolite_all_taxas_adj_diversity_annotated_fdr = all_all_assoc_adj_diversity_annotated_fdr[all_all_assoc_adj_diversity_annotated_fdr['metabolite_id'].isin(numpy.unique(sig_metab_assoc))]

all_all_assoc_adj_diversity_annotated_fdr.to_csv('./assoc_results/{0}_all_metabolite_all_{1}_{3}_div_adj_fdr_corr_{2}.csv'.format(cohort, taxa_level, bmi_or_not, classification), index = None)
sig_metabolite_all_taxas_adj_diversity_annotated_fdr.to_csv('./assoc_results/{0}_sig_metabolite_all_{1}_{3}_div_adj_fdr_corr_{2}.csv'.format(cohort, taxa_level, bmi_or_not, classification), index = None)
print(len(all_all_assoc_adj_diversity_annotated_fdr))
print(len(all_all_assoc_adj_diversity_annotated_fdr['metabolite_id'].unique()))
print(len(sig_metabolite_all_taxas_adj_diversity_annotated_fdr))
print(len(sig_metabolite_all_taxas_adj_diversity_annotated_fdr['metabolite_id'].unique()))

964
326
213
114


# microbiome - protein analysis

In [ ]:
# get phenotype and covariates
prot_pheno = pandas.read_csv('../gxe/input_pheno/{0}_prot_10202019.pheno'.format(cohort), sep = '\t', index_col = [0,1])
prot_covar = pandas.read_csv('../gxe/input_pheno/{0}_prot_10202019.covar'.format(cohort), sep = '\t', index_col = [0,1])
proteins = prot_pheno.columns

# merge phenotype data with covariates
prot_covariates = ['age', 'age_sq','sex_age', 'sex_age_sq', 'sex', 'Chip_ID_CVD2', 'Chip_ID_CVD3', 'Chip_ID_INF', 'PC1', 'PC2','PC3','PC4','PC5', 'bmi']
prot_factors = ['sex','Chip_ID_CVD2', 'Chip_ID_CVD3', 'Chip_ID_INF']
prot_combined = prot_pheno.join(prot_covar[prot_covariates])
prot_combined.index = prot_combined.index.droplevel(0)

# get list of proteins
cvd2_proteins = prot_combined.columns[prot_combined.columns.str.contains('CVD2_')].values
cvd3_proteins = prot_combined.columns[prot_combined.columns.str.contains('CVD3_')].values
inf_proteins = prot_combined.columns[prot_combined.columns.str.contains('INF_')].values
all_proteins = numpy.concatenate([cvd2_proteins, cvd3_proteins, inf_proteins])

# get ONLY significant associations
proteomics_metadata = adi.get_snapshot('proteomics_metadata', path='/homedir/notebooks-cheng/snapshots/arivale_snapshot_2019-05-20_1330/')
proteomics_metadata['name'] = proteomics_metadata['name'].str.replace(',','_').str.replace('-','_')
proteomics_metadata['gene_name'] = proteomics_metadata['gene_name'].str.replace(',','_').str.replace('-','_')
proteomics_metadata['panel_gene'] = proteomics_metadata.apply(lambda x: x['panel'] + '_' + x['gene_name'], axis = 1)
proteomics_metadata_dict = proteomics_metadata.set_index('name')['panel_gene'].to_dict()

sig_prot_assoc = pandas.read_csv('/homedir/notebooks-cheng/mr_project/metab_prot/mr_analysis/validated_proteomics_association.csv')
sig_prot_assoc = sig_prot_assoc[sig_prot_assoc['Pr(>|t|)'] <= 1e-4]
sig_prot_assoc = numpy.unique(sig_prot_assoc['Phenotype'].astype(str).values)

In [ ]:
# adjust for covariates
prot_adj = prot_combined[[]].copy()
R.r.assign('prot_combined',prot_combined)

for category in prot_factors:
    R.r('prot_combined${0} <- factor(prot_combined${0})'.format(category))

for prot in cvd2_proteins:
    R.r('fit <- lm({0} ~ age + age_sq + sex_age + sex_age_sq + sex + bmi + Chip_ID_CVD2 + PC1 + PC2 + PC3 + PC4 + PC5, data=prot_combined, na.action=na.exclude)'.format(prot))
    prot_adj[prot] = R.r('residuals(fit)')

for prot in cvd3_proteins:
    R.r('fit <- lm({0} ~ age + age_sq + sex_age + sex_age_sq + sex + bmi + Chip_ID_CVD3 + PC1 + PC2 + PC3 + PC4 + PC5, data=prot_combined, na.action=na.exclude)'.format(prot))
    prot_adj[prot] = R.r('residuals(fit)')

for prot in inf_proteins:
    R.r('fit <- lm({0} ~ age + age_sq + sex_age + sex_age_sq + sex + bmi + Chip_ID_INF + PC1 + PC2 + PC3 + PC4 + PC5, data=prot_combined, na.action=na.exclude)'.format(prot))
    prot_adj[prot] = R.r('residuals(fit)')

In [ ]:
# merge adjusted phenotypes
micro_prot_subset_pheno = micro_pheno[micro_pheno.index.isin(prot_pheno.index)]
micro_prot_adj = micro_prot_subset_pheno.join(prot_adj)
micro_prot_adj.to_csv('./joint_pheno/{0}_{1}_micro_prot_adj.pheno'.format(cohort, taxa_level))

micro_prot_unadj = micro_prot_subset_pheno.join(prot_pheno)
micro_prot_unadj.to_csv('./joint_pheno/{0}_{1}_micro_prot_unadj.pheno'.format(cohort, taxa_level))
prot_covar.loc[prot_covar.index.isin(micro_prot_unadj.index),:].to_csv('./joint_pheno/{0}_{1}_micro_prot_unadj.covar'.format(cohort, taxa_level))
print(len(micro_prot_adj))

## association analysis

### run OLS with all protein - taxa combo

In [ ]:
all_protein_all_taxas_assoc = []
for protein in numpy.unique(all_proteins):
    for taxa in taxas:
        temp = micro_prot_adj[[protein, taxa]].dropna()
        
        ols_results = sm.OLS(temp[protein].values, sm.add_constant(temp[taxa].values)).fit()
        cons_pval, pval = ols_results.pvalues
                
        all_protein_all_taxas_assoc.append([protein, taxa, pval])

print('num of sig proteins:{0}'.format(len(numpy.unique(all_proteins))))        
print('num of taxas: {0}'.format(len(taxas)))
print('num of comparisons:{0}'.format(len(all_protein_all_taxas_assoc)))

In [ ]:
all_protein_all_taxas_assoc_df = pandas.DataFrame(all_protein_all_taxas_assoc).drop_duplicates()
all_protein_all_taxas_assoc_df.columns = ['protein_id','taxa_id','pval']
all_protein_all_taxas_assoc_annotated = all_protein_all_taxas_assoc_df.merge(micro_metadata)
all_protein_all_taxas_assoc_annotated['protein'] = list(map(proteomics_metadata_dict.get, all_protein_all_taxas_assoc_annotated['protein_id'].values))

all_protein_all_taxas_assoc_within_fam_bonff = all_protein_all_taxas_assoc_annotated.loc[all_protein_all_taxas_assoc_annotated['pval'] <= 0.05/len(taxas),]
all_protein_all_taxas_assoc_within_fam_bonff = all_protein_all_taxas_assoc_within_fam_bonff[['protein_id','taxa_id','pval', 'taxonomy','protein']]
all_protein_all_taxas_assoc_within_fam_bonff.to_csv('./assoc_results/{0}_all_protein_all_{1}_assoc_within_fam_corr_{2}.csv'.format(cohort, taxa_level, bmi_or_not), index = None)
len(all_protein_all_taxas_assoc_within_fam_bonff)

In [12]:
sig_protein_all_taxas_assoc_annotated = all_protein_all_taxas_assoc_annotated[all_protein_all_taxas_assoc_annotated['protein_id'].isin(numpy.unique(sig_prot_assoc))]
sig_protein_all_taxas_assoc_annotated.to_csv('./assoc_results/{0}_sig_protein_all_{1}_assoc_asv_{2}.csv'.format(cohort, taxa_level, bmi_or_not), index = None)

sig_protein_all_taxas_assoc_within_fam_bonff = sig_protein_all_taxas_assoc_annotated.loc[sig_protein_all_taxas_assoc_annotated['pval'] <= 0.05/len(taxas),]
sig_protein_all_taxas_assoc_within_fam_bonff = sig_protein_all_taxas_assoc_within_fam_bonff[['protein_id','taxa_id','pval', 'taxonomy','protein']]
sig_protein_all_taxas_assoc_within_fam_bonff.to_csv('./assoc_results/{0}_sig_protein_all_{1}_assoc_within_fam_corr_{2}.csv'.format(cohort, taxa_level, bmi_or_not), index = None)
len(sig_protein_all_taxas_assoc_within_fam_bonff)

74